# Folium to visualize data

* Este material está basado en las siguientes fuentes:
    1. [Domino_Data_Lab](https://blog.dominodatalab.com/creating-interactive-crime-maps-with-folium/) 
    2. [Python_GIS](https://automating-gis-processes.github.io/site/notebooks/L5/interactive-map-folium.html)


* Cuáles son los **argumentos** que más vamos a utilizar?
    1. **location**
    2. **popup**
    3. **icon**

# Pequeño ejemplo con 3 puntos. 

In [173]:
import folium
from folium.plugins import MarkerCluster


m = folium.Map(location=[44, -73], zoom_start=5)

marker_cluster = MarkerCluster().add_to(m)
m

# Agregar markers de manera independiente

In [174]:
folium.Marker(
    location=[40.67, -73.94],
    popup="Add popup text here.",
    icon=folium.Icon(color="green", icon="ok-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -73.94],
    popup="Add popup text here.",
    icon=folium.Icon(color="red", icon="remove-sign"),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -71.94],
    popup="Add popup text here.",
    icon=None,
).add_to(marker_cluster)

m

# Vamos a crear 100 puntos alrededor del mundo

In [177]:
import numpy as np

size = 100
lons = np.random.randint(-180, 180, size=size)
lats = np.random.randint(-90, 90, size=size)


In [179]:
# Creamos una lista con tuplas
locations = list(zip(lats, lons))

In [181]:
# Vamos a colocar info de locacion en cada popup
popups = ["lon:{}<br>lat:{}".format(lon, lat) for (lat, lon) in locations]


In [182]:
m = folium.Map(
    location=[np.mean(lats), np.mean(lons)], tiles="Cartodb Positron", zoom_start=1
)

marker_cluster = MarkerCluster(
    locations=locations,
    popups=popups,
    name="1000 clustered icons",
    overlay=True,
    control=True,
    
)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m

# Agregar markers usando Loops

In [183]:
%%time

# Crear al mapa o plataforma en la que irán los markers
m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

# Crear la caracteristica del marker
marker_cluster = MarkerCluster(
    name='1000 clustered icons',
    overlay=True,
    control=False,
    icon_create_function=None
)

# Agregar uno a uno las locaciones sobre el marker
for k in range(size):
    location = lats[k], lons[k]
    marker = folium.Marker(location=location)
    popup = 'lon:{}<br>lat:{}'.format(location[1], location[0])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

# Agregar todos los markers sobre el mapa 
marker_cluster.add_to(m)

#folium.LayerControl().add_to(m)
m

Wall time: 10.7 ms


# Si no te gusta esperdar = > Fastermarker!!! 

Usando FastMarkerCluster es posible renderizar miles de puntos mucho más rápido que la clase MarkerCluster.

In [128]:
from folium.plugins import FastMarkerCluster

In [185]:
%%time


m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

FastMarkerCluster(data=list(zip(lats, lons))).add_to(m)

folium.LayerControl().add_to(m)
m

Wall time: 4.99 ms


# HEATMAPS

In [186]:
from folium.plugins import HeatMap

# Create a Map instance
m = folium.Map(location=[ np.mean(lats), np.mean(lons)], tiles = 'stamentoner', zoom_start=1, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(locations).add_to(m)

# Alternative syntax:
#m.add_child(HeatMap(points_array, radius=15))

# Show map
m

# Trabajemos con data real!!!

# Data de insitutos Superiores Pedagógicos del Perú

In [188]:
institutos = pd.read_csv( "institutos1.csv" )

In [135]:
institutos.head()

,cod_mod,ltimoden_metaatencion,cuentadeid_postulante_procesoadm,sumaden_flagingresante,sumaden_flagmatriculado,ratio,ratio1,dif,dif1,nlat_ie,...,population_ccpp,altitude,Dblock1km,Dblock2km,Dblock3km,Dblock5km,Dblock10km,Dblock20km,Dblock30km,_merge
0,207597,307,481,302,301,0.980456,1.566775,-6,174,-13.52107,...,106400.0,3386.0,14840369,742184,494123,296073,148036,7418,4912,matched (3)
1,207613,240,502,204,204,0.850000,2.091667,-36,262,-13.63997,...,45864.0,2342.0,14970470,748235,499156,299094,149047,7423,4915,matched (3)
2,239970,537,1047,484,476,0.886406,1.949721,-61,510,-15.47827,...,216716.0,3831.0,17000163,850081,566054,340032,170016,8508,5605,matched (3)
3,262311,423,526,353,349,0.825059,1.243499,-74,103,-6.23041,...,23202.0,2358.0,6781025,339512,226341,135205,67102,3351,2234,matched (3)
4,273979,425,544,411,405,0.952941,1.280000,-20,119,-6.49004,...,67362.0,309.0,7070857,353428,235285,141171,70085,3542,2328,matched (3)


In [189]:
locations = list(zip(institutos["nlat_ie"], institutos["nlong_ie"]))
postu_ingres = list(zip(institutos["cuentadeid_postulante_procesoadm"], institutos["sumaden_flagingresante"]))
locations

[(-13.521070000000002, -71.97748),
 (-13.639970000000002, -72.88261),
 (-15.478270000000002, -70.12756),
 (-6.23041, -77.86942999999998),
 (-6.4900400000000005, -76.36539000000002),
 (-14.05115, -75.75387000000002),
 (-9.9344, -76.23690000000002),
 (-18.01575, -70.24829),
 (-5.1810599999999996, -80.66289),
 (-11.2539, -74.637),
 (-7.16051, -78.512),
 (-6.85847, -78.1452),
 (-9.52118, -77.52506),
 (-13.169929999999999, -74.19741000000002),
 (-12.78588, -74.97243),
 (-6.76202, -79.83548999999998),
 (-6.5636, -78.6485),
 (-15.87551, -69.99554),
 (-4.912, -80.6947),
 (-17.21399, -70.94074000000002),
 (-13.66356, -73.34892000000002),
 (-14.1019, -72.7019),
 (-10.66411, -76.25261999999998),
 (-6.03121, -76.96302),
 (-8.38598, -74.5686),
 (-14.1424, -71.40813),
 (-14.688089999999999, -74.12099000000002),
 (-13.301779999999999, -72.12163000000001),
 (-5.8982, -76.1294),
 (-9.14094, -77.74014),
 (-8.822699999999998, -77.46271999999999),
 (-12.9344, -74.2487),
 (-12.04935, -77.03283999999998),
 

# Crear cluster Marker para el número de institutos por regiones

In [190]:
m = folium.Map(
    location=[ -12.0431800, -77.02824], tiles="Cartodb Positron", zoom_start=4
)

popups = ["Número de postulantes:{}<br>Número de ingresantes:{}".format(postu, ingres) for (postu, ingres) in postu_ingres]
popups

['Número de postulantes:481<br>Número de ingresantes:302',
 'Número de postulantes:502<br>Número de ingresantes:204',
 'Número de postulantes:1047<br>Número de ingresantes:484',
 'Número de postulantes:526<br>Número de ingresantes:353',
 'Número de postulantes:544<br>Número de ingresantes:411',
 'Número de postulantes:461<br>Número de ingresantes:426',
 'Número de postulantes:813<br>Número de ingresantes:391',
 'Número de postulantes:520<br>Número de ingresantes:327',
 'Número de postulantes:430<br>Número de ingresantes:316',
 'Número de postulantes:112<br>Número de ingresantes:112',
 'Número de postulantes:1311<br>Número de ingresantes:932',
 'Número de postulantes:1170<br>Número de ingresantes:580',
 'Número de postulantes:1630<br>Número de ingresantes:754',
 'Número de postulantes:1152<br>Número de ingresantes:516',
 'Número de postulantes:1357<br>Número de ingresantes:525',
 'Número de postulantes:897<br>Número de ingresantes:804',
 'Número de postulantes:967<br>Número de ingresant

In [191]:
marker_cluster = MarkerCluster(
    locations=locations,
    name="1000 clustered icons",
    overlay=True,
    control=True,
    popups=popups
    
)



marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m

# Creamos Heatmap para el número de institutos en el Perú

In [192]:
from folium.plugins import HeatMap
# Create a Map instance
m = folium.Map(location=[ -12.0431800, -77.02824], tiles = 'stamentoner', zoom_start=4, control_scale=True)

# Add heatmap to map instance
# Available parameters: HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True)
HeatMap(locations).add_to(m)

# Alternative syntax:
#m.add_child(HeatMap(points_array, radius=15))

# Show map
m

# Circles Marker , Adjusting radius 

In [194]:

folium_map = folium.Map(location=[-9.1899672, -75.015152], zoom_start=7)
folium_map
for index, row in institutos.iterrows():
        # calculate net departures
        num_postul =    row["cuentadeid_postulante_procesoadm"]/50
        
        # radius of circles
        radius = num_postul
        
        # choose the color of the marker
        if num_postul>0:
            # color="#FFCE00" # orange
            # color="#007849" # green
            color="#0A8A9F" # teal
            
        else:
            # color="#0375B4" # blue
            # color="#FFCE00" # yellow  
            color="#E37222" # tangerine
            
        
        # add marker to the map
        folium.CircleMarker(location=(row["nlat_ie"],
                                      row["nlong_ie"]),
                            radius=radius,
                            color=color).add_to(folium_map)
folium_map

In [ ]:
folium_map